<a href="https://colab.research.google.com/github/shelbygreen/leoncounty/blob/master/site_suitability_parks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Site Suitability Analysis Model for determining where the engineering project should be constructed.

In [0]:
#import libraries
import shapefile
from shapely.geometry import shape
import geopandas as gpd
import pandas as pd
import requests
import matplotlib.pyplot as plt
import os 
import folium

#read the shapefile into the gdf variable (geodataframe)
gdf = gpd.read_file("./LC_Owned_Land.shp")

In [0]:
# create df for parks SA

use_filters = gdf.USE == 'Parks and Recreation'
status_filters = gdf.STATUS == 'Park'
building_filters = gdf.NOBLDGS == 0 

pdf = gdf[use_filters & status_filters & building_filters]

In [0]:
# manipulating the dfs to drop the variables unneeded
pdf = pdf.drop(['TAXID', 'TAXID_LABE', 'CALC_ACREA', 'OWNER2', 'OWNER1A', 'PAGENUM',
          'LGL_ISSUE', 'NOTES', 'ZONING', 'Shape_Leng', 'Shape_Area', 'LEASE_INFO',
          'DEVELOPMNT', 'FLD_CAT', 'PURCHASED', 'BOOKNUM'], axis=1).reset_index()

In [0]:
#SITE SUITABILITY ANALYSIS
"""
census variables:
percent black, pct_black: 100 * B2001_003E/B02001_001E
percent with internet, pct_internet: 100 * B28002_002E/B28002_001E
percent 65 and older with disability and no health coverage: B18135_029E/B18135_029E
median age, median_age: B01002_001E
median income, median_income: B19013_001E
percent of households in poverty, pct_in_poverty: 100 * B17017_002E/B17017_001E
percent in labor force (employed), pct_employed: 100 * B23025_005E/B23025_003E
"""

#grabbing census data
host, dataset = "https://api.census.gov/data", "acs/acs5"
year = "2018"
variables = ["B17017_001E", "B17017_002E", "B23025_003E", "B23025_005E",
             "B02001_003E", "B02001_001E", "B28002_002E", "B28002_001E",
             "B18135_029E", "B18135_024E", "B01002_001E", "B19013_001E"]

for variable in variables:
    url = "/".join([host, year, dataset])
    get_vars = ["NAME"] + variables
    predicates = {}
    predicates["get"] = ",".join(get_vars)
    predicates["for"] = "tract:*"
    predicates["in"] = "state:12;county:073"
    r = requests.get(url, params=predicates)
    df = pd.DataFrame(columns=r.json()[0], data=r.json()[1:])

In [0]:
#merging leon county census tracts w/ census data
tdf = gpd.read_file("./usa_tracts.shp")[['STATEFP','COUNTYFP', 'TRACTCE', 'geometry']] \
    .rename(index=str, 
            columns={'STATEFP': 'state', 
                     'COUNTYFP':'county',
                     'TRACTCE': 'tract'})

tdf = tdf.merge(df).set_index(['state', 'county'])

#modifying the dataframe
tdf = tdf[tdf["B19013_001E"].astype(int) > 0] #removes income < $0

#making the ID variables
tdf["pct_black"] = 100 * tdf["B02001_003E"].astype(int)/tdf["B02001_001E"].astype(int)
tdf["pct_has_internet"] = 100 * tdf["B28002_002E"].astype(int)/tdf["B28002_001E"].astype(int)
#tdf["pct_disabled"] = 100 * tdf["B18135_029E"].astype(int)/tdf["B18135_029E"].astype(int)
tdf["median_age"] = tdf["B01002_001E"].astype(float)
tdf["median_income"] = tdf["B19013_001E"].astype(int)
tdf["pct_in_poverty"] = 100 * tdf["B17017_002E"].astype(int)/tdf["B17017_001E"].astype(int)
tdf["pct_unemployed"] = 100 * tdf["B23025_005E"].astype(int)/tdf["B23025_003E"].astype(int)

In [0]:
'''
criteria for the model:
--parks--
park have to be in (or near) a census tract with the following:
1. low WiFi accessibility (< 51% have access to the internet)
2. younger population (< 30 years old)
3. older population (> 50 years old)
4. low income (median income < $30,000 & pct_in_poverty > 30%)
5. high unemployment rate (> 10%) 

--facilities--
done externally via flood and solar assessment
'''
#parks - site selection
tdf = tdf.assign(
    internet_criteria = lambda tdf: tdf['pct_has_internet'].map(lambda pct_has_internet: 1 if pct_has_internet < 70 else 0),
    age1_criteria = lambda tdf: tdf['median_age'].map(lambda median_age: 1 if median_age < 30 else 0),
    age2_criteria = lambda tdf: tdf['median_age'].map(lambda median_age: 1 if median_age > 50 else 0),
    income_criteria = lambda tdf: tdf['median_income'].map(lambda median_income: 1 if median_income < 30000 else 0),
    poverty_criteria = lambda tdf: tdf['pct_in_poverty'].map(lambda pct_in_poverty: 1 if pct_in_poverty > 30 else 0),
    unemployment_criteria = lambda tdf: tdf['pct_unemployed'].map(lambda pct_unemployed: 1 if pct_unemployed > 10 else 0))

tdf["potential_site"] = tdf.iloc[:, 21:27].sum(axis=1)

In [0]:
#save df as geoJSON
tdf.to_file('./tdf.geojson', driver='GeoJSON')
pdf.to_file('./pdf.geojson', driver = 'GeoJSON')

In [0]:
#INTERACTIVE MODEL OF MODEL RESULTS

fm = folium.Map([30.4361696,-84.2959047], zoom_start=10, tiles='cartodbpositron')
                           
#score layer
folium.Choropleth(
    geo_data = './tdf.geojson',
    data = tdf,
    columns = ['tract', 'potential_site'],
    fill_color = 'YlGn',
    key_on = 'feature.properties.tract',
    name = 'tracts', 
    legend_name = 'Site Score',
    control = True,
    overlay = True,
    show = True
).add_to(fm)
                           
folium.GeoJson(
    data = pdf,
    name = "parks",
    overlay = True,
    control = True,
    show = True).add_child(folium.GeoJsonTooltip(fields=['NAME'])).add_to(fm)

folium.LayerControl().add_to(fm)

fm


Ideas so far:

*   using neighborhood vulnerability to pinpoint community center/park for self-sufficient food forest
*   using neighborhood vulnerability and disability metrics to pinpoint community center/park for targeted upgrades to increase accesibility (wifi, use)
*   using neighborhood vulnerability to target UHI reduction efforts
*   using neighborhood vulnerability to influence placement of next (pocket) park